In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from scipy.integrate import solve_ivp
import numpy as np
import matplotlib.pyplot as plt
plt.style.use(r'~/PaperDoubleFig.mplstyle')

def rhs(t, y):
    """
    The right-hand side of the 1D canonical FEL equations;
    t - the current time;
    y - array of [A, theta, eta]
    """
    n = len(y)//2
    A = y[0]
    theta = y[1:n+1]
    eta = y[n+1:]
    dA_dt = np.mean(np.exp(-1j*theta))
    dtheta_dt = eta
    deta_dt = -2*np.real(A*np.exp(1j*theta))
    return np.concatenate(([dA_dt],
                           dtheta_dt,
                           deta_dt))
"""
sol = solve_ivp(rhs, [0, 4*np.pi],
                np.concatenate(([A0+0j], theta0, eta0)),
                max_step=0.1)
"""

## Plotting functions

In [ ]:
def plot(sol):
    ne = len(sol.y)//2
    plt.semilogy(sol.t, np.abs(sol.y[0])**2)
    plt.xlabel(r"Normalized undulator coordinate, $\tau$")
    plt.ylabel(r"Normalized power (log scale), $|A|^2$")
    plt.xlim([0, tau])
    plt.ylim([1e-8, 1.4])
    plt.twinx()
    plt.plot(sol.t, np.abs(sol.y[0])**2, 'r')
    plt.ylabel(r"Normalized power (lin scale), $|A|^2$")
    plt.ylim([0, 1.4])
    plt.show()
    plt.plot(sol.t, np.real(sol.y[1:ne+1,:].T)/np.pi)
    plt.xlim([0, tau])
    plt.xlabel(r"Normalized undulator coordinate, $\tau$")
    plt.ylabel(r"Phase, $\theta_n/\pi$")
    plt.show()

    plt.plot(sol.t, np.real(sol.y[ne+1:,:].T))
    plt.xlim([0, tau])
    plt.xlabel(r"Normalized undulator coordinate, $\tau$")
    plt.ylabel(r"Energy detuning, $\eta_n$")
    plt.twinx()
    plt.plot(sol.t, np.mean(np.real(sol.y[ne+1:,:].T), axis=1), 'black',linestyle='-.', linewidth=3)
    plt.ylabel(r"Averaged energy detuning, $\bar{\eta}$")
    plt.show()

## Naive start

In [ ]:
ne = 100
tau = 20
A0 = 0 
theta0 = np.linspace(-4*np.pi, 4*np.pi, ne)
p0 = np.zeros(ne)
sol = solve_ivp(rhs, [0, tau], np.concatenate(([A0+0j], theta0, p0)), max_step=0.1)
plot(sol)

In [ ]:
ne = 100
tau = 20
A0 = 0 
theta0 = np.linspace(-4*np.pi, 4*np.pi, ne, endpoint=False)
p0 = np.zeros(ne)
sol = solve_ivp(rhs, [0, tau], np.concatenate(([A0+0j], theta0, p0)), max_step=0.1)
plot(sol)

## Seeded FEL

In [ ]:
ne = 100
tau = 20
A0 = 1e-3 
theta0 = np.linspace(-4*np.pi, 4*np.pi, ne, endpoint=False)
p0 = np.zeros(ne)
sol = solve_ivp(rhs, [0, tau], np.concatenate(([A0+0j], theta0, p0)), max_step=0.1)
plot(sol)
A0 = 1e-3 
t = sol.t[:100]
E = A0*(np.exp((1j+np.sqrt(3))*t/2)
            + np.exp((1j-np.sqrt(3))*t/2)
            + np.exp(-1j*t)
            )/3.
plt.semilogy(t, np.abs(E)**2)
plt.show()

In [ ]:
ne = 100
tau = 5*np.pi
theta0 = np.linspace(-4*np.pi, 4*np.pi, ne, endpoint=False)
p0 = np.zeros(ne)
for A0 in np.sqrt([1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1]):
    sol = solve_ivp(rhs, [0, tau], np.concatenate(([A0+0j], theta0, p0)), max_step=0.1)
    plt.semilogy(sol.t, np.abs(sol.y[0])**2)
plt.xlabel(r"Normalized undulator coordinate, $\tau$")
plt.ylabel(r"Normalized power (log scale), $|A|^2$")
plt.xlim([0, tau])
plt.ylim([1e-8, 1.8])
plt.show()

## SASE operation

In [ ]:
ne = 100
tau = 5*np.pi
A0 = 0 
p0 = np.zeros(ne)
theta0 = np.random.uniform(low=-4*np.pi, high=4*np.pi, size=ne)
sol = solve_ivp(rhs, [0, tau], np.concatenate(([A0+0j], theta0, p0)), max_step=0.1)
plot(sol)

In [ ]:
ne = 100
tau = 10
A0 = 0 
p0 = np.zeros(ne)
for n in np.arange(100):
    theta0 = np.random.uniform(low=-4*np.pi, high=4*np.pi, size=ne)
    sol = solve_ivp(rhs, [0, tau], np.concatenate(([A0+0j], theta0, p0)), max_step=0.1)
    plt.plot(sol.t, np.abs(sol.y[0])**2)
plt.xlabel(r"Normalized undulator coordinate, $\tau$")
plt.ylabel(r"Normalized power (lin scale), $|A|^2$")
plt.xlim([0, tau])
plt.ylim([1e-8, 1.8])
plt.show()
for n in np.arange(100):
    theta0 = np.random.uniform(low=-4*np.pi, high=4*np.pi, size=ne)
    sol = solve_ivp(rhs, [0, tau], np.concatenate(([A0+0j], theta0, p0)), max_step=0.1)
    plt.semilogy(sol.t, np.abs(sol.y[0])**2)
plt.xlabel(r"Normalized undulator coordinate, $\tau$")
plt.ylabel(r"Normalized power (log scale), $|A|^2$")
plt.xlim([0, tau])
plt.ylim([1e-8, 1.8])
plt.show()

## Energy spread

In [ ]:
ne = 100
tau = 5*np.pi
A0 = 0 
theta0 = np.linspace(-4*np.pi, 4*np.pi, ne, endpoint=False)
p0 = np.random.normal(0, 0.1, ne)
sol = solve_ivp(rhs, [0, tau], np.concatenate(([A0+0j], theta0, p0)), max_step=0.1)
plot(sol)

In [ ]:
ne = 100
tau = 10
A0 = 0 
theta0 = np.linspace(-4*np.pi, 4*np.pi, ne, endpoint=False)
for n in np.arange(100):
    p0 = np.random.normal(0, 0.1, ne)
    sol = solve_ivp(rhs, [0, tau], np.concatenate(([A0+0j], theta0, p0)), max_step=0.1)
    plt.plot(sol.t, np.abs(sol.y[0])**2)
plt.xlabel(r"Normalized undulator coordinate, $\tau$")
plt.ylabel(r"Normalized power (lin scale), $|A|^2$")
plt.xlim([0, tau])
plt.ylim([1e-8, 1.8])
plt.show()
for n in np.arange(100):
    p0 = np.random.normal(0, 0.1, ne)
    sol = solve_ivp(rhs, [0, tau], np.concatenate(([A0+0j], theta0, p0)), max_step=0.1)
    plt.semilogy(sol.t, np.abs(sol.y[0])**2)
plt.xlabel(r"Normalized undulator coordinate, $\tau$")
plt.ylabel(r"Normalized power (log scale), $|A|^2$")
plt.xlim([0, tau])
plt.ylim([1e-8, 1.8])
plt.show()

## Controlled SASE

In [ ]:
ne = 100
tau = 5*np.pi
A0 = 0
dtheta = 2/np.sqrt(100)
theta0 = np.linspace(-4*np.pi, 4*np.pi, ne, endpoint=False)
theta0 -= dtheta*np.sin(theta0)
p0 = np.zeros(ne)
sol = solve_ivp(rhs, [0, tau], np.concatenate(([A0+0j], theta0, p0)), max_step=0.1)
plot(sol)

## Shot noise power

In [ ]:
ne = 25
tau = 5*np.pi
A0 = 0
dtheta = 1.38e-3
theta0 = np.linspace(-4*np.pi, 4*np.pi, ne, endpoint=False)
theta0 -= dtheta*np.sin(theta0)
p0 = np.zeros(ne)
sol = solve_ivp(rhs, [0, tau], np.concatenate(([A0+0j], theta0, p0)), max_step=0.1)
plot(sol)

In [ ]:
ne = 25
tau = 5*np.pi
A0 = 1.38e-3/2
dtheta = 0
theta0 = np.linspace(-4*np.pi, 4*np.pi, ne, endpoint=False)
theta0 -= dtheta*np.sin(theta0)
p0 = np.zeros(ne)
sol = solve_ivp(rhs, [0, tau], np.concatenate(([A0+0j], theta0, p0)), max_step=0.1)
plot(sol)

In [ ]:
ne = 25
tau = 4*np.pi
A0 = 0
p0 = np.zeros(ne)
dtheta = 1.38e-3
theta0 = np.linspace(-4*np.pi, 4*np.pi, ne, endpoint=False)
theta0 -= dtheta*np.sin(theta0)
sol = solve_ivp(rhs, [0, tau], np.concatenate(([A0+0j], theta0, p0)), max_step=0.1)
plt.semilogy(sol.t, np.abs(sol.y[0])**2, label='SASE')

A0 = 1.38e-3/2
dtheta = 0
theta0 = np.linspace(-4*np.pi, 4*np.pi, ne, endpoint=False)
theta0 -= dtheta*np.sin(theta0)
sol = solve_ivp(rhs, [0, tau], np.concatenate(([A0+0j], theta0, p0)), max_step=0.1)
plt.semilogy(sol.t, np.abs(sol.y[0])**2, 'r', label='shot-noise')
plt.xlabel(r"Normalized undulator coordinate, $\tau$")
plt.ylabel(r"Normalized power (log scale), $|A|^2$")
plt.xlim([0, tau])
plt.ylim([1e-8, 1.8])
plt.legend()
plt.show()

## Resonant condition

In [ ]:
ne = 25
tau = 10*np.pi
A0 = 1.38e-3/2
dtheta = 0
theta0 = np.linspace(-4*np.pi, 4*np.pi, ne, endpoint=False)
theta0 -= dtheta*np.sin(theta0)
res = []
eta = np.linspace(-2,2.3)
for eta0 in eta:
    p0 = eta0*np.ones(ne)
    sol = solve_ivp(rhs, [0, tau],
                    np.concatenate(([A0+0j], theta0, p0)),
                    max_step=0.1)
    res.append(np.max(np.abs(sol.y[0])**2))
plt.plot(eta, res, 'o-')
plt.xlabel(r'Energy detuning, $\eta_0$')
plt.ylabel(r'Peak power, $max(|A|^2)$')
plt.xlim([-2, 2.3])
plt.ylim([0, 4])
plt.show()

In [ ]:
def saseFit(sol):
    peak = np.max(np.abs(sol.y[0])**2)
    ipeak = np.argmax(np.abs(sol.y[0])**2)
    iStart = np.argmin((sol.t-2)**2)
    iStop = np.argmin((np.abs(sol.y[0][iStart:ipeak])**2-0.9*peak)**2)
    x = sol.t[iStart:iStop]
    y = np.abs(sol.y[0][iStart:iStop])**2
    z = np.polyfit(x, np.log(y), 1)  # fit parameters
    return 9.*np.exp(z[1]), z[0]

In [ ]:
ne = 25
tau = 4*np.pi
A0 = 1.38e-3/2
dtheta = 0
theta0 = np.linspace(-4*np.pi, 4*np.pi, ne, endpoint=False)
theta0 -= dtheta*np.sin(theta0)
res = []
eta = np.linspace(-6,1.93, 50)
for eta0 in eta:
    p0 = eta0*np.ones(ne)
    sol = solve_ivp(rhs, [0, tau],
                    np.concatenate(([A0+0j], theta0, p0)),
                    max_step=0.1)
    try:
        res.append(saseFit(sol)[1])
    except:
        break
plt.plot(eta[:len(res)], res/np.sqrt(3), '.-')
plt.xlabel(r'Energy detuning, $\eta_0$')
plt.ylabel(r'$\Gamma/\sqrt{3}$')
plt.ylim([0,1.01])
plt.show()

- Energy spread does not reduce the gain as expected for some reason;

In [ ]:
ne = 25
tau = 4*np.pi
A0 = 1.38e-3/2
dtheta = 0
theta0 = np.linspace(-4*np.pi, 4*np.pi, ne, endpoint=False)
theta0 -= dtheta*np.sin(theta0)
res = []
eta = np.linspace(-6,1.93, 50)
for eta0 in eta:
    p0 = np.random.normal(eta0, 0.01, ne)
    sol = solve_ivp(rhs, [0, tau],
                    np.concatenate(([A0+0j], theta0, p0)),
                    max_step=0.1)
    try:
        res.append(saseFit(sol)[1])
    except:
        break
plt.plot(eta[:len(res)], res/np.sqrt(3), '.-')
plt.xlabel(r'Energy detuning, $\eta_0$')
plt.ylabel(r'$\Gamma/\sqrt{3}$')
plt.ylim([0,1.01])
plt.show()